In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from funciones import *
from unidecode import unidecode


In [10]:
load_dotenv()  # Carga las variables del archivo .env

api_key = os.getenv("API_TOKEN")
sql_password = os.getenv("SQL_PASSWORD")

In [2]:
df_tailandia = pd.read_csv("animales_marinos_tailandia_2015_2025.csv")
df_indonesia = pd.read_csv("animales_marinos_indonesia_2015_2025.csv")
df_filipinas = pd.read_csv("animales_marinos_filipinas_2015_2025.csv")

In [3]:
df_tailandia = clean_common_name(df_tailandia)
df_indonesia = clean_common_name(df_indonesia)
df_filipinas = clean_common_name(df_filipinas)

In [29]:
df_tailandia = fix_fishe(df_tailandia)
df_filipinas = fix_fishe(df_filipinas)
df_indonesia = fix_fishe(df_indonesia)

In [30]:
df_tailandia['country'] = 'Thailand'
df_indonesia['country'] = 'Indonesia'
df_filipinas['country'] = 'Philippines'

In [31]:
df_tailandia=df_tailandia.drop_duplicates()
df_indonesia=df_indonesia.drop_duplicates()
df_filipinas=df_filipinas.drop_duplicates()

In [32]:
df_tailandia.drop(['year', 'month'], axis=1, inplace=True)
df_indonesia.drop('year', axis=1, inplace=True)

In [33]:
df_tailandia['common_name'] = df_tailandia['common_name'].fillna(df_tailandia['scientific_name'])
df_indonesia['common_name'] = df_indonesia['common_name'].fillna(df_indonesia['scientific_name'])
df_filipinas['common_name'] = df_filipinas['common_name'].fillna(df_filipinas['scientific_name'])

In [34]:
df_tailandia = df_tailandia[df_tailandia['scientific_name'] != 'Actinopterygii']
df_tailandia = df_tailandia[df_tailandia['common_name'] != 'shark']

df_indonesia = df_indonesia[df_indonesia['scientific_name'] != 'Actinopterygii']
df_indonesia = df_indonesia[df_indonesia['common_name'] != 'shark']

df_filipinas = df_filipinas[df_filipinas['scientific_name'] != 'Actinopterygii']
df_filipinas = df_filipinas[df_filipinas['common_name'] != 'shark']

In [35]:
df_filipinas[df_filipinas['common_name'].str.contains('Nan', na=False)]

,date,scientific_name,common_name,latitude,longitude,country


In [36]:
df_total = pd.concat([df_filipinas, df_tailandia, df_indonesia], ignore_index=True)

In [37]:
df_species_unique = (
    df_total[['scientific_name', 'common_name']]
    .drop_duplicates(subset='scientific_name', keep='first')
    .reset_index(drop=True)
)

In [ ]:
# 🔐 Token de iNaturalist
TOKEN = os.getenv("API_TOKEN")
headers = {"Authorization": TOKEN}

In [ ]:
#  Ejecutar la búsqueda y construir columna nueva
image_urls = []

for i, row in df_species_unique.iterrows():
    sci = row['scientific_name']
    com = row['common_name']
    print(f"🔍 Buscando imagen para: {sci}...", end=" ")
    url = get_image_url(sci, com)
    image_urls.append(url)
    print("✅" if url else "❌")
    time.sleep(0.2)  # para no saturar la API

# ➕ Añadir columna al DataFrame
df_species_unique['image_url'] = image_urls

# 💾 Guardar como CSV
df_species_unique.to_csv('species_with_images.csv', index=False)

🔍 Buscando imagen para: Mullidae... ✅
🔍 Buscando imagen para: Upeneus... ✅
🔍 Buscando imagen para: Inegocia japonica... ✅
🔍 Buscando imagen para: Pleuronectoidei... ✅
🔍 Buscando imagen para: Diagramma pictum... ✅
🔍 Buscando imagen para: Dendrochirus zebra... ✅
🔍 Buscando imagen para: Lactoria fornasini... ✅
🔍 Buscando imagen para: Paragobiodon xanthosoma... ✅
🔍 Buscando imagen para: Opistognathus randalli... ✅
🔍 Buscando imagen para: Taenianotus triacanthus... ✅
🔍 Buscando imagen para: Paraluteres prionurus... ✅
🔍 Buscando imagen para: Hippocampus denise... ✅
🔍 Buscando imagen para: Paragobiodon modestus... ✅
🔍 Buscando imagen para: Solenostomus paradoxus... ✅
🔍 Buscando imagen para: Ophichthus bonaparti... ✅
🔍 Buscando imagen para: Acanthochromis polyacanthus... ✅
🔍 Buscando imagen para: Pseudanthias hypselosoma... ✅
🔍 Buscando imagen para: Cirrhilabrus cyanopleura... ✅
🔍 Buscando imagen para: Helcogramma striata... ✅
🔍 Buscando imagen para: Valenciennea helsdingenii... ✅
🔍 Buscando i

In [38]:
df_species_unique = pd.read_csv('species_with_images.csv')

In [3]:
df_total[df_total['common_name'].str.contains('nan', na=False)]

NameError: name 'df_total' is not defined

In [48]:
df_species_unique[df_species_unique['common_name'].str.contains('nan', na=False)]

,scientific_name,common_name,image_url
1126,Heniochus chrysostomus,pennant bannerfish,https://inaturalist-open-data.s3.amazonaws.com...
1310,Synanceia,synanceia,https://inaturalist-open-data.s3.amazonaws.com...
2415,Nandus,nandu,https://inaturalist-open-data.s3.amazonaws.com...
2474,Betta pugnax,penang betta,https://inaturalist-open-data.s3.amazonaws.com...
2757,Schistura menanensis,schistura menanensi,https://static.inaturalist.org/photos/17550517...
3016,Acanthurus tennentii,lieutenant surgeonfish,https://inaturalist-open-data.s3.amazonaws.com...
3103,Nandus oxyrhynchus,nandus oxyrhynchu,https://inaturalist-open-data.s3.amazonaws.com...
3216,Hemimyzon nanensis,nan hillstream loach,https://static.inaturalist.org/photos/24820098...
3344,Devario annandalei,devario annandalei,https://static.inaturalist.org/photos/31641308...
4863,Tominanga aurea,tominanga aurea,No image available


In [43]:
df_species_unique = clean_common_name(df_species_unique)
df_species_unique = fix_fishe(df_species_unique)
df_total = clean_common_name(df_total)
df_total = fix_fishe(df_total)

In [47]:
df_species_unique['image_url'] = df_species_unique['image_url'].fillna("No image available")

In [51]:
df_total = df_total.merge(
    df_species_unique[['scientific_name', 'image_url']],
    on='scientific_name',
    how='left'
)

In [52]:
df_species_unique = df_species_unique.reset_index(drop=True)
df_species_unique['fish_id'] = df_species_unique.index + 1

In [53]:
for df in [df_tailandia, df_indonesia, df_filipinas, df_total, df_species_unique]:
    df['scientific_name'] = df['scientific_name'].str.strip().str.title()
    df['common_name'] = df['common_name'].str.strip().str.title()

In [55]:
df_tailandia = df_tailandia.merge(df_species_unique[['scientific_name', 'fish_id', 'image_url']], on='scientific_name', how='left')
df_indonesia = df_indonesia.merge(df_species_unique[['scientific_name', 'fish_id', 'image_url']], on='scientific_name', how='left')
df_filipinas = df_filipinas.merge(df_species_unique[['scientific_name', 'fish_id', 'image_url']], on='scientific_name', how='left')

In [56]:
for df in [df_tailandia, df_indonesia, df_filipinas,df_species_unique, df_total]:
    df.rename(columns={'image_url': 'image'}, inplace=True)

In [57]:
df_country = pd.concat([df_tailandia, df_indonesia, df_filipinas], ignore_index=True)[['fish_id', 'country']].drop_duplicates()

In [60]:
df_final = pd.concat([df_tailandia, df_filipinas, df_indonesia], ignore_index=True)

In [61]:
df_final['Year'] = pd.to_datetime(df_final['date']).dt.year

In [62]:
df_final['Month'] = pd.to_datetime(df_final['date']).dt.month

In [64]:
for df in [df_final, df_tailandia, df_indonesia, df_filipinas, df_species_unique, df_country]:
    if 'common_name' in df.columns and 'scientific_name' in df.columns:
        df.loc[df['common_name'] == 'Nan', 'common_name'] = df['scientific_name']

In [65]:
df_final.to_csv("marine_biodiversity_final.csv", index=False)

In [66]:
df_final.to_csv("marine_biodiversity_final.csv", index=False)
df_tailandia.to_csv("thailand.csv", index=False)
df_indonesia.to_csv("indonesia.csv", index=False)
df_filipinas.to_csv("philippines.csv", index=False)
df_species_unique.to_csv("fish_species.csv", index=False)
df_country.to_csv("fish_country.csv", index=False)

In [ ]:
usuario = "root"           
contraseña = os.getenv("SQL_PASSWORD")  
host = "localhost"
puerto = 3306
base_de_datos = "marine_biodiversity"

# Crear motor de conexión con SQLAlchemy
engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}:{puerto}/{base_de_datos}")

In [68]:
df_species = pd.read_csv("fish_species.csv")
df_country = pd.read_csv("fish_country.csv")
df_thailand = pd.read_csv("thailand.csv")
df_indonesia = pd.read_csv("indonesia.csv")
df_philippines = pd.read_csv("philippines.csv")

In [78]:
df_species.to_sql('fish_species', con=engine, if_exists='append', index=False)

5273

In [79]:
df_country.to_sql('fish_country', con=engine, if_exists='append', index=False)

9325

In [80]:
df_thailand.to_sql('thailand', con=engine, if_exists='append', index=False)

42248

In [81]:
df_indonesia.to_sql('indonesia', con=engine, if_exists='append', index=False)

163475

In [82]:
df_philippines.to_sql('philippines', con=engine, if_exists='append', index=False)

45139